In [1]:
import tensorflow as tf
import numpy as np

import keras
import pickle
import h5py
import time
import gzip

from tensorflow.python.ops import data_flow_ops
from tensorflow.keras.layers import Embedding

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load the input vectors

In [0]:
with gzip.open('./drive/My Drive/CoE202TermProject/datasets/train.chunk.pickle', 'rb') as f:
    traindata = pickle.load(f)

train_product   = traindata['product'][:]
train_label   = traindata['label'][:]

with gzip.open('./drive/My Drive/CoE202TermProject/datasets/valid.chunk.pickle', 'rb') as f:
    validdata = pickle.load(f)

valid_product   = validdata['product'][:]
valid_label   = validdata['label'][:]

In [0]:
batch_size = 512
embd_size  = 1024
voca_size  = 100001
lr         = 1e-4
n_epochs   = 3
valid_freq = 3
max_seq    = len(train_product[0][4])
n_classes  = np.max(train_label) + 1

### Placeholders

In [0]:
products   = tf.placeholder(dtype=tf.float32, shape=[None, max_seq], name='products')
product_counts = tf.placeholder(dtype=tf.float32, shape=[None, max_seq], name='product_counts')
brands = tf.placeholder(dtype=tf.float32, shape=[None], name='brands')
models = tf.placeholder(dtype=tf.float32, shape=[None, max_seq], name='models')
model_counts = tf.placeholder(dtype=tf.float32, shape=[None, max_seq], name='model_counts')
makers = tf.placeholder(dtype=tf.float32, shape=[None], name='makers')
labels   = tf.placeholder(dtype=tf.int32, shape=[None], name='labels')
is_train = tf.placeholder(dtype=tf.bool, name='is_train')

### Deep neural network

In [6]:
# Define embedding
embd = Embedding(voca_size, embd_size, name='product_embd')                     # Define embedding function


# Product embedding
product_embd = embd(products)                                                 # Get the embedding vector of products
product_count_shaped = tf.reshape(product_counts, (-1, max_seq, 1))
product_embd_mat = tf.keras.layers.dot([product_embd, product_count_shaped], axes=1) # Multiply the counts of products to the corresponding embedding vectors
product_embd = tf.reshape(product_embd_mat, (-1, 1, embd_size))
product_out = tf.layers.dropout(product_embd, training=is_train)

# Brand embedding
brands_out = embd(tf.reshape(brands, (-1, 1)))

# Model embedding
model_embd = embd(models)
model_count_shaped = tf.reshape(model_counts, (-1, max_seq, 1))
model_embd_mat = tf.keras.layers.dot([model_embd, model_count_shaped], axes=1)
model_embd = tf.reshape(model_embd_mat, (-1, 1, embd_size))
model_out = tf.layers.dropout(model_embd, training=is_train)

# Makers embedidng
makers_out = embd(tf.reshape(makers, (-1, 1)))

# Combining product and models
combined = tf.keras.layers.concatenate([brands_out, model_out, makers_out, product_out], axis=1)
bn = tf.layers.batch_normalization(combined, training=is_train)                 # Batch normalization
relu = tf.nn.relu(bn)                                                           # ReLU function

x = tf.layers.dense(relu, n_classes)
x = tf.layers.batch_normalization(x, training=is_train)
x = tf.nn.relu(x)
x = tf.layers.flatten(x)

x = tf.layers.dense(x, n_classes)
x = tf.layers.batch_normalization(x, training=is_train)
x = tf.nn.relu(x)

logits = tf.layers.dense(x, n_classes)

# Softmax
preds = tf.nn.softmax(logits, name='predictions')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.flatten instead.


### Loss funciton & Optimizer

In [7]:
# Softmax cross entropy loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(labels, n_classes), logits=logits)

# Weight decay
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([loss] + reg_losses, name='total_loss')

# Optimizer
optm = tf.train.AdamOptimizer(lr)
train_op = optm.minimize(loss, global_step=tf.train.get_global_step(), name='step_update')
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
train_op = tf.group([train_op, update_ops])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Accuracy

In [0]:
top1_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=1)
top1_acc = tf.identity(top1_acc, name='top1_acc')

top5_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=5)
top5_acc = tf.identity(top5_acc, name='top5_acc')

### Batch generator

In [0]:
def generator(mode='training'):
    if mode == 'training':
        n_data = len(train_label)
        indices = np.arange(n_data)
        np.random.shuffle(indices)
        
        for start_idx in range(0, n_data, batch_size):
            if start_idx + batch_size <= n_data:
                excerpt = indices[start_idx: start_idx + batch_size]
                yield list(zip(*train_product[excerpt, :])), train_label[excerpt]

    elif mode == 'valid':
        n_data = len(valid_label)
        indices = np.arange(n_data)

        for start_idx in range(0, n_data, batch_size):
            if start_idx + batch_size <= n_data:
                excerpt = indices[start_idx: start_idx + batch_size]
                yield list(zip(*valid_product[excerpt, :])), valid_label[excerpt]

### Training session

In [10]:
import tqdm
import math

with tf.Session() as sess:
    tic = time.time()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        print("\n\nEpoch {0:03d} / {1:03d}\n".format(epoch, n_epochs))
        training_loss = []
        for b_product, b_label in tqdm.tqdm(
            generator(mode='training'),
            total=math.ceil(len(train_label) / batch_size)
        ):
            feed_dict = {products  : b_product[4],
                         product_counts: b_product[5],
                         brands: b_product[0],
                         models: b_product[1],
                         model_counts: b_product[2],
                         makers: b_product[3],
                         labels    : b_label,
                         is_train  : True}

            _, train_loss = sess.run([train_op, loss], feed_dict=feed_dict)
            training_loss.append(train_loss)

        toc = time.time()
        print("[*] TRAIN Loss {0:.4f} | Time {1:.2f}s".format(np.mean(training_loss), toc - tic))
        
        if (epoch+1) % valid_freq == 0:
            TOP1, TOP5 = [], []
            for b_product, b_label in generator(mode='valid'):
                feed_dict = {products  : b_product[4],
                        product_counts: b_product[5],
                        brands: b_product[0],
                        models: b_product[1],
                        model_counts: b_product[2],
                        makers: b_product[3],
                        labels    : b_label,
                        is_train  : False}
                    
                t1_acc, t5_acc = sess.run([top1_acc, top5_acc], feed_dict=feed_dict)
                TOP1.append(t1_acc)
                TOP5.append(t5_acc)
            print("[*] VALIDATION Top-1 Acc: {0:.4f} | Top-5 Acc: {1:.4f}".format(np.mean(TOP1), np.mean(TOP5)))

    saver.save(sess, './drive/My Drive/CoE202TermProject/models/models')

  0%|          | 0/1563 [00:00<?, ?it/s]



Epoch 000 / 003



  0%|          | 1/1563 [00:00<02:58,  8.77it/s]

[*] TRAIN Loss 3.4028 | Time 155.26s


Epoch 001 / 003



  0%|          | 1/1563 [00:00<02:58,  8.75it/s]

[*] TRAIN Loss 1.3468 | Time 309.03s


Epoch 002 / 003



100%|█████████▉| 1562/1563 [02:33<00:00, 10.18it/s]


[*] TRAIN Loss 0.7763 | Time 462.53s
[*] VALIDATION Top-1 Acc: 0.7541 | Top-5 Acc: 0.9215


## Test process
### Load the test data

In [0]:
# test data
with gzip.open('./drive/My Drive/CoE202TermProject/datasets/test.chunk.pickle', 'rb') as f:
    testdata = pickle.load(f)

test_product  = testdata['product'][:]
pids          = testdata['pids'][:]

### Batch generator for test dataset

In [0]:
def generator(mode='test'):
    if mode == 'test':
        n_data = len(test_product)
        indices = np.arange(n_data)
        
        for start_idx in range(0, n_data, batch_size):
            excerpt = indices[start_idx: start_idx + batch_size]
            yield list(zip(*test_product[excerpt, :]))

In [13]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./drive/My Drive/CoE202TermProject/models/models.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./drive/My Drive/CoE202TermProject/models/'))
    DNN = tf.get_default_graph()

    preds = []
    for b_product in generator(mode='test'):
        feed_dict = {DNN.get_tensor_by_name('products:0'): b_product[4],
                     DNN.get_tensor_by_name('product_counts:0'): b_product[5],
                     DNN.get_tensor_by_name('brands:0'): b_product[0],
                     DNN.get_tensor_by_name('models:0'): b_product[1],
                     DNN.get_tensor_by_name('model_counts:0'): b_product[2],
                     DNN.get_tensor_by_name('makers:0'): b_product[3],
                     DNN.get_tensor_by_name('is_train:0')     : False}

        pred = sess.run(DNN.get_tensor_by_name('predictions:0'), feed_dict=feed_dict)
        preds.extend(pred)


INFO:tensorflow:Restoring parameters from ./drive/My Drive/CoE202TermProject/models/models


### Save the submission files

In [0]:
# Indexing of predictions
argpreds = np.argmax(preds, axis=1)

# Load label dictionary
with open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle', 'rb') as f:
    y_dict = pickle.load(f)
# y_dict = pickle.loads(open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle').read())

# Inverse label dictionary
inv_y_dict = dict((y,x) for x,y in y_dict.items())
submissions = [inv_y_dict[argpred] for argpred in argpreds]

# Write the results to 'submissions.csv'
f = open('./drive/My Drive/CoE202TermProject/submissions.csv', 'w')
for i, j in zip(pids, submissions):
    line = '{},{}\n'.format(i,j)
    f.write(line)
f.close()

You should submit the 'submissions.csv' file and 4 tf.save files ('checkpoint', 'dnn_models.data', 'dnn_models.index', 'dnn_models.meta') in models folder